In [1]:
import numpy as np
import pandas as pd
from random import *
from pandas import Series

In [2]:
spel_index = {' ':0, 'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,'n':14,'o':15,'p':16,\
              'q':17,'r':18,'s':19,'t':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26}

In [3]:
# txt 파일 읽기 (trigger word)

r = open('./data/trigger_word.txt', mode='rt', encoding='utf-8')
word = r.read()
# word = "friend"

In [4]:
# 알파벳 number indexing, 총 길이 15 zero padding, 

def indexing(lst_word):
    if len(lst_word) < 16:
        index_word = np.zeros(shape=(15,),dtype=int)
        index_word = index_word.tolist()
        j = 0
        for i in lst_word:
            index_word[j] = spel_index[i]
            j += 1
        return index_word
    else:
        print("input under word length 15")
        return None

In [5]:
# indexing 해서 배열로 만들기

def to_index(listed):
    indexed = []
    for i in listed:
        indexed.append(indexing(i))
    indexed = np.array(indexed)
    return indexed

In [6]:
# 한자씩 빼기

def one_miss(word):
    word_list = []
    for i in range(0, len(word)):
        w = list(word)
        w.pop(i)
        w.append(" ")
        word_list.append(w)
    word_list = np.array(word_list)
    return word_list

In [7]:
# 두자씩 빼기

def two_miss(word):
    word_list = []
    r = int(len(word) * (len(word) - 1) / 2)    # nC2
    
    while len(word_list) < r:
        w = list(word)
        a = randrange(0, len(word))
        b = randrange(0, len(word) - 1)
        w.pop(a)
        w.pop(b)
        if w not in word_list:    
            word_list.append(w)
    word_list = np.array(word_list)
            
    return word_list

In [8]:
# 기본 단어 + one_miss()

def default_word(word):
    word_list = np.array([list(word)])
    word_list = np.vstack((word_list, one_miss(word)))
    
    return word_list

In [9]:
# 랜덤하게 한자 추가하는 노이즈

def make_noise(word_list):
    a = word_list.tolist()
    
    for i in range(0, len(a)):
        x = randrange(0, len(a[0]))
        y = randrange(0, 27)

        a[i].insert(x, list(spel_index.keys())[y])
    a = np.array(a)
    return a

In [10]:
# change spelling

def change_spel(data, spel_from, spel_to):
    changed = np.where(data == spel_index[spel_from], spel_index[spel_to], data)
    data = np.append(data, changed, axis=0)
    data = np.unique(data, axis=0)
    
    return data

In [11]:
def make_true_data(word):
    default_list = default_word(word)  # default
    noise_word = make_noise(default_list)  # default 에서 노이즈
    
    # indexing
    indexed_d = to_index(default_list)  
    indexed_n = to_index(noise_word)
    if len(word) < 6:
        d_t_data = indexed_d
    else:
        two_miss_word = two_miss(word)  # 두자 빼기
        indexed_t = to_index(two_miss_word)
        d_t_data = np.vstack((indexed_d, indexed_t))
        
    #  모음 변환 a (1), e (5), i(9), o(15), u(21)
    change_a_e = change_spel(d_t_data, "a", "e")
    change_a_i = change_spel(d_t_data, "a", "i")
    change_a_o = change_spel(d_t_data, "a", "o")
    change_a_u = change_spel(d_t_data, "a", "u")
    change_a = np.vstack((change_a_e, change_a_i, change_a_o, change_a_u))

    change_e_a = change_spel(d_t_data, "e", "a")
    change_e_i = change_spel(d_t_data, "e", "i")
    change_e_o = change_spel(d_t_data, "e", "o")
    change_e_u = change_spel(d_t_data, "e", "u")
    change_e = np.vstack((change_e_a, change_e_i, change_e_o, change_e_u))
    
    change_i_a = change_spel(d_t_data, "i", "a")
    change_i_e = change_spel(d_t_data, "i", "e")
    change_i_o = change_spel(d_t_data, "i", "o")
    change_i_u = change_spel(d_t_data, "i", "u")
    change_i = np.vstack((change_i_a, change_i_e, change_i_o, change_i_u))

    change_o_a = change_spel(d_t_data, "o", "a")
    change_o_e = change_spel(d_t_data, "o", "e")
    change_o_i = change_spel(d_t_data, "o", "i")
    change_o_u = change_spel(d_t_data, "o", "u")
    change_o = np.vstack((change_o_a, change_o_e, change_o_i, change_o_u))

    change_u_a = change_spel(d_t_data, "u", "a")
    change_u_e = change_spel(d_t_data, "u", "e")
    change_u_i = change_spel(d_t_data, "u", "i")
    change_u_o = change_spel(d_t_data, "u", "o")
    change_u = np.vstack((change_u_a, change_u_e, change_u_i, change_u_o))
    
    d_t_data = np.vstack((change_a, change_e, change_i, change_o, change_u))
    d_t_data = np.unique(d_t_data, axis=0)
    
    # 합치기
    true_data = np.vstack((d_t_data, indexed_n))
    
    # label 넣어주기
    true_label = np.ones(shape=(true_data.shape[0],1),dtype=int)
    true_data = np.hstack((true_data, true_label))
    
    return true_data    

In [12]:
true_data = make_true_data(word)
print(true_data.shape)

(132, 16)


In [13]:
def to_list(text_word):
    listed = []
    for i in text_word:
        listed.append(list(i))
    return listed

In [14]:
# false data

false = pd.read_csv("./data/others.txt",sep='\n', names=["word"])

def false_data(data):
    data.word = data.word.str.lower()
    if word in data.word.values:
        f_index = data.word[data.word == word].index[0]
        false_word = data.word.drop(f_index)
    else:
        false_word = data.word
    return false_word

In [15]:
false_word = false_data(false)
false_data_indexed = to_index(to_list(false_word))

In [16]:
# True 데이터에서 생성된 데이터 중 False 데이터에 있는 것 삭제

false_data_word = false_data_indexed.tolist()
true_data_word = np.delete(true_data, -1, 1).tolist()

for i in true_data_word:
    if i in false_data_word:
        d_index = false_data_word.index(i)
        false_data_word.pop(d_index)
        
false_data_word = np.array(false_data_word)

In [17]:
false_label = np.zeros(shape=(false_data_word.shape[0],1),dtype=int)
false_data = np.hstack((false_data_word, false_label))

In [18]:
np.random.shuffle(true_data)
np.random.shuffle(false_data)

train_true = true_data[:-20]
test_true = true_data[-20:]

np.random.shuffle(false_data)
train_false = false_data[:-750]
test_false = false_data[-750:]


In [19]:
train_data = np.vstack((train_true, train_false))
test_data = np.vstack((test_true, test_false))

In [20]:
np.random.shuffle(train_data)
np.random.shuffle(test_data)

In [21]:
X_train = np.delete(train_data,-1,1)
y_train = train_data[:,-1:]

In [22]:
X_test = np.delete(test_data,-1,1)
y_test = test_data[:,-1:]

In [23]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', C=8, gamma=0.1)
 
svm_model.fit(X_train, y_train)

C:\Users\zxwlg\miniconda3\envs\text\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=8, gamma=0.1)

In [24]:
svm_model.score(X_test, y_test)

0.9623376623376624

In [25]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_test_pred = svm_model.predict(X_test)
cf = confusion_matrix(y_test, y_test_pred)
print(cf)

[[733  17]
 [ 12   8]]


In [26]:
y_train_pred = svm_model.predict(X_train)
cf = confusion_matrix(y_train, y_train_pred)
print(cf)

[[2238    1]
 [   0  112]]


In [27]:
a = 'friend'
a = indexing(a)

test_svm = svm_model.predict(np.array([a]))
test_svm

array([1])

In [28]:
## augmentation 된 data 출력

data = np.delete(true_data,-1,1)

def data_txt(data, file_name):
    spel_index_s = Series(spel_index, dtype=np.int)
    list_data = spel_index_s.index
    list_name = spel_index_s.values
    num_to_spel = Series(data = list_data, index = list_name)
    
    data_l = data.tolist()
    
    for i in data_l:
        for j in range(len(i)):
            i[j] = num_to_spel[i[j]]
            
    f = open("{}.txt".format(file_name), 'w')
    for i in data_l:
        for j in range(len(i)):
            f.write(i[j])
        f.write("\n")
    f.close()

In [29]:
data_txt(data, "true_data")